In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
cus = pd.read_csv('../data/customers.csv')

C:\Users\Bowen\AppData\Local\Temp\ipykernel_10164\3306847811.py:1: DtypeWarning: Columns (8,14,17,18) have mixed types. Specify dtype option on import or set low_memory=False.
  cus = pd.read_csv('../data/customers.csv')
